<a href="https://colab.research.google.com/github/futugyou/pyproject/blob/master/google_colab/text_generation_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required dependencies
%pip install langchain-community langchain-google-genai langchain google-generativeai
%pip install duckduckgo-search

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", api_key=userdata.get("GOOGLE_API_KEY")
)

In [ ]:
from langchain.prompts import PromptTemplate

# 1. react prompt
react_template = """You are a helpful agent. You can use tools to solve the task.

Available tools:
{tools}

Protocol (case-sensitive):
- After every "Thought:" you must EITHER:
  a) output an "Action:" (one of [{tool_names}]) followed by "Action Input:", then wait for an "Observation:", OR
  b) output "Final Answer:" to finish.
- Never write another "Thought:" after "Final Answer:".
- If no tool is needed, skip directly to "Final Answer:".

Use this exact format:

Question: {input}

{agent_scratchpad}
# The scratchpad above may contain prior Thought/Action/Action Input/Observation steps.
# Continue from there.

Thought: <what to do next>
Action: <one of [{tool_names}]>
Action Input: <input to the action>
Observation: <result of the action>
... (repeat Thought/Action/Action Input/Observation as needed)
Thought: I now know the final answer
Final Answer: <the final answer to the original question>
"""


prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"],
)

In [ ]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="a web search engine. use this to as a search engine for general queries.",
    func=search.run,
)

tools = load_tools(["llm-math"], llm=llm)
tools.append(search_tool)

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

# ReAct agent
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
agent_executor.invoke(
    {
        "input": "what is the current price of a MacBook Pro in usd? how much would it cost in eur if the exchange rate is 0.85 eur for 1 usd."
    }
)